In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
import tensorflow as tf


In [ ]:
base_dir="/content/drive/MyDrive/mini_project dataset/dataset"

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

# Pre-processing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

validation_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 1150 images belonging to 4 classes.
Found 126 images belonging to 4 classes.


In [ ]:
# create base model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 128)     147584    
                                                        

In [ ]:
history=model.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
36/36 [==============================] - 234s 6s/step - loss: 1.9187 - accuracy: 0.2809 - val_loss: 1.3441 - val_accuracy: 0.5714
Epoch 2/5
36/36 [==============================] - 29s 785ms/step - loss: 1.1124 - accuracy: 0.5391 - val_loss: 0.9495 - val_accuracy: 0.6746
Epoch 3/5
36/36 [==============================] - 32s 886ms/step - loss: 0.8805 - accuracy: 0.6643 - val_loss: 0.8048 - val_accuracy: 0.7143
Epoch 4/5
36/36 [==============================] - 34s 934ms/step - loss: 0.8350 - accuracy: 0.6809 - val_loss: 0.7968 - val_accuracy: 0.6825
Epoch 5/5
36/36 [==============================] - 31s 857ms/step - loss: 0.7734 - accuracy: 0.6965 - val_loss: 0.7778 - val_accuracy: 0.6984


In [ ]:
# VGG 16 Model
from tensorflow.keras.applications import VGG16
model_vgg16 = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in model_vgg16.layers:
  layer.trainable=False

model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
for layer in model_vgg16.layers:
  layer.trainable=False

model_vgg16.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model = Sequential()
model.add(model_vgg16)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 27562308 (105.14 MB)
Trainable params: 12847620 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
36/36 [==============================] - 38s 842ms/step - loss: 2.1165 - accuracy: 0.5470 - val_loss: 0.9203 - val_accuracy: 0.5873
Epoch 2/5
36/36 [==============================] - 26s 716ms/step - loss: 0.7062 - accuracy: 0.7252 - val_loss: 0.6297 - val_accuracy: 0.7302
Epoch 3/5
36/36 [==============================] - 29s 820ms/step - loss: 0.5816 - accuracy: 0.7765 - val_loss: 0.6032 - val_accuracy: 0.7698
Epoch 4/5
36/36 [==============================] - 29s 801ms/step - loss: 0.5172 - accuracy: 0.8113 - val_loss: 0.5573 - val_accuracy: 0.7857
Epoch 5/5
36/36 [==============================] - 29s 793ms/step - loss: 0.4734 - accuracy: 0.8191 - val_loss: 0.4937 - val_accuracy: 0.7937


In [ ]:
model.save('vgg_classifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
vg = tf.keras.models.load_model('vgg_classifier.h5')

In [ ]:
vg.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 27562308 (105.14 MB)
Trainable params: 12847620 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
import cv2
import numpy as np
img = cv2.imread('/content/drive/MyDrive/mini_project dataset/rock.jpg')
img = cv2.resize(img, (224,224))
img = np.reshape(img, [1,224,224,3])
img = img/255

preds = vg.predict(img)

max_idx = np.argmax(preds)
class_indices = {v:k for k,v in train_generator.class_indices.items()}
class_indices[max_idx]

1/1 [==============================] - 1s 1s/step


'rocky'